In [1]:
from typing import Callable
import numpy as np

In [20]:
def objective(x):
    return (x-3)**2

def objective2(x):
    y = -x**4 + 4*x**3 + 30*x**2 - 50*x + 200
    return y

## 1.0 Funções de bracketing 

### 1.1 Bracket Minimum

In [3]:
def bracket_minimum(f: Callable, x=0, s=1e-3, k=2.0):
    """
    
    Args:
        - f (Callable): função objetiva
        - x (float): ponto inicial de partida
        - s (float): hiperparâmetro de avanço (valor padrão: 1e-3)
        - k (float): hiperparâmetro multiplicador do avanço (valor padrão: 2)
    Returns:
        - tuple (float, float): tupla com os pontos da extremidades
    """
    
    a, y_a = x, f(x)                        # ponto a: x e f(x)
    b, y_b = a+s, f(a+s)                    # ponto b: x+s e f(x+s)
    
    # se f(x+s) > f(x), então inverte-se os pontos e valores
    if y_b > y_a:
        a, b = b, a
        y_a, y_b = y_b, y_a
        s = -s    # inverte o sinal
    # loop para 
    while True: 
        c, y_c = b+s, f(b+s)
        if y_c > y_b:
                return (a, c) if a < c else (c, a)
        a, y_a, b, y_a = b, y_b, c, y_c
        s*=k

# Testa função
f = objective
x1, x2 = bracket_minimum(f, x=2)
print(f'Método: bracket_minium - O valor mínimo da função f = (x-3)**2 está no intervalo {x1:.2f} a {x2:.2f}')

Método: bracket_minium - O valor mínimo da função f = (x-3)**2 está no intervalo 2.51 a 4.05


### 1.2 Fibonacci Search

In [4]:
def fibonacci_search(f: Callable, a: float, b: float, n: int, e=1e-2):
    """
    Executa a busca Fibonacci para encontrar um intervalo onde um mínimo (ou máximo) de uma função unimodal pode estar 
    localizado.

    Args:
       - f (Callable): A função unimodal a ser otimizada.
       - a (float): O limite inferior do intervalo inicial de busca.
       - b (float): O limite superior do intervalo inicial de busca.
       - n (int): O número de iterações a serem realizadas na busca.
       - e (float, optional): A precisão desejada para a busca. O valor padrão é 1e-2.

    Returns:
        - Tuple[float, float]: Uma tupla contendo os limites inferior e superior do intervalo onde o mínimo (ou máximo) 
        está localizado.
    """
    
    s = (1-np.sqrt(5))/(1+np.sqrt(5))
    phi = (1+np.sqrt(5))/2
    rho = 1/(phi*(1-s**(n+1))/(1-s**n))
    d = rho*b + (1-rho)*a
    y_d = f(d)
    for i in range(n-1):
        if i == n - 1:
            c = e*a + (1-e)*d
        else: 
            c = rho*a + (1-rho)*b
        y_c = f(c)
        if y_c < y_d:
            b, d, y_d = d, c, y_c
        else:
            a, b = b, c
        rho = 1/(phi*(1-s**(n-i+1))/(1-s**(n-i)))
    return (a, b) if (a < b) else (b, a)

# Testa função
f = objective
x1, x2 = fibonacci_search(f, a=0, b=10, n=10)
print(f'Método: fibonacci_search - O valor mínimo da função f = (x-3)**2 está no intervalo {x1:.2f} a {x2:.2f}')

Método: fibonacci_search - O valor mínimo da função f = (x-3)**2 está no intervalo 2.92 a 3.06


### 1.3 Golden Section Search

In [6]:
def golden_section_search(f, a: float, b: float, n: int):
    phi = (1+np.sqrt(5))/2
    rho = phi - 1
    d = rho*b + (1-rho)*a
    y_d = f(d)
    for _ in range(n-1):
        c = rho*a + (1-rho)*b
        y_c = f(c)
        if y_c < y_d:
            b, d, y_d = d, c, y_c
        else:
            a, b = b, c
    return (a, b) if (a < b) else (b, a)

# Testa função
f = objective
x1, x2 = golden_section_search(f, a=0, b=10, n=10)
print(f'Método: golden_section_search - O valor mínimo da função f = (x-3)**2 está no intervalo {x1:.2f} a {x2:.2f}')

Método: golden_section_search - O valor mínimo da função f = (x-3)**2 está no intervalo 2.92 a 3.05


### 1.3 Quadratic Fit Search

In [10]:
def quadratic_fit_search(f, a: float, b: float, c: float, n: int):
    ya, yb, yc = f(a), f(b), f(c)
    for _ in range(1,n-3):
        x = 0.5*(ya*(b**2-c**2) + yb*(c**2-a**2) + yc*(a**2-b**2))/(ya*(b-c) + yb*(c-a) + yc*(a-b))
        yx = f(x)
        if x > b:
            if yx > yb:
                c, yc = x, yx
            else:
                a, ya, b, yb = b, yb, x, yx
        elif x < b:
            if yx > yb:
                a, ya = x, yx
            else:
                c, yc, b, yb = b, yb, x, yx
    return (a, b, c)    

In [23]:
f = objective2
a, b, c = quadratic_fit_search(f, a=-5, b=2, c=10, n=10)
print(f'Método: quadratic_fit_search - O valor mínimo da função f = -x**4 + 4*x**3 + 30*x**2 - 50*x + 200 é próximo de {b:.2f}')

Método: quadratic_fit_search - O valor mínimo da função f = -x**4 + 4*x**3 + 30*x**2 - 50*x + 200 é próximo de 0.75
